In [1]:
import zipfile
import os
import pandas as pd

In [2]:
zip_path='/content/water.zip'
extract_path='/content/water'

In [3]:
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# List the extracted files
os.listdir(extract_path)

['dataset']

In [4]:
# List files inside the "dataset" subfolder
dataset_folder = os.path.join(extract_path, "dataset")
os.listdir(dataset_folder)

['sample_submission.csv', 'test.csv', 'train.csv']

In [5]:
# Load the data
train_df = pd.read_csv(os.path.join(dataset_folder, "train.csv"))
test_df = pd.read_csv(os.path.join(dataset_folder, "test.csv"))
sample_submission_df = pd.read_csv(os.path.join(dataset_folder, "sample_submission.csv"))

In [6]:
# Display basic info for each dataframe
train_info = train_df.info()
test_info = test_df.info()
sample_submission_head = sample_submission_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14000 entries, 0 to 13999
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Timestamp                 14000 non-null  object 
 1   Residents                 14000 non-null  int64  
 2   Apartment_Type            13574 non-null  object 
 3   Temperature               13559 non-null  float64
 4   Humidity                  14000 non-null  object 
 5   Water_Price               14000 non-null  float64
 6   Period_Consumption_Index  14000 non-null  float64
 7   Income_Level              13574 non-null  object 
 8   Guests                    14000 non-null  int64  
 9   Amenities                 8003 non-null   object 
 10  Appliance_Usage           13585 non-null  float64
 11  Water_Consumption         14000 non-null  float64
dtypes: float64(5), int64(2), object(5)
memory usage: 1.3+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6000 entr

In [7]:
# Show first few rows of the training data
train_head = train_df.head()

(train_info, test_info, sample_submission_head, train_head)

(None,
 None,
        Timestamp  Water_Consumption
 0  11/10/2014 16             300.05
 1  12/10/2014 00             179.45
 2  12/10/2014 08              78.55
 3  12/10/2014 16             121.81
 4  13/10/2014 00             125.85,
        Timestamp  Residents Apartment_Type  Temperature Humidity  Water_Price  \
 0  01/01/2002 00          1         Studio        15.31    46.61         1.06   
 1  01/01/2002 08          4            NaN        21.01    66.11         2.98   
 2  01/01/2002 16          2        Cottage        12.86    60.86         1.44   
 3  02/01/2002 00          2           1BHK        20.16    50.58         1.48   
 4  02/01/2002 08          2        Cottage        16.23    52.25         1.14   
 
    Period_Consumption_Index  Income_Level  Guests      Amenities  \
 0                      0.97           Low       0  Swimming Pool   
 1                      0.91  Upper Middle       1  Swimming Pool   
 2                      1.43        Middle       0            

# **Processing of data**

In [8]:
# Step-by-step immediate cleaning tasks

# 1. Convert 'Humidity' from object to float
train_df['Humidity'] = pd.to_numeric(train_df['Humidity'], errors='coerce')
test_df['Humidity'] = pd.to_numeric(test_df['Humidity'], errors='coerce')

In [9]:

# 2. Convert 'Timestamp' to datetime
train_df['Timestamp'] = pd.to_datetime(train_df['Timestamp'], format="%d/%m/%Y %H")
test_df['Timestamp'] = pd.to_datetime(test_df['Timestamp'], format="%d/%m/%Y %H")

In [10]:

# 3. Extract features from Timestamp: hour, day of week, month
for df in [train_df, test_df]:
    df['Hour'] = df['Timestamp'].dt.hour
    df['DayOfWeek'] = df['Timestamp'].dt.dayofweek
    df['Month'] = df['Timestamp'].dt.month

In [11]:
import pandas as pd

# Check missing values
missing_train = train_df.isnull().sum()
missing_test = test_df.isnull().sum()

# Combine into a DataFrame for easy viewing
missing_df = pd.DataFrame({
    "Train Missing": missing_train,
    "Test Missing": missing_test
})

# Display the missing values table
print(missing_df)


                          Train Missing  Test Missing
Amenities                          5997        2513.0
Apartment_Type                      426         166.0
Appliance_Usage                     415         177.0
DayOfWeek                             0           0.0
Guests                                0           0.0
Hour                                  0           0.0
Humidity                            397         194.0
Income_Level                        426         165.0
Month                                 0           0.0
Period_Consumption_Index              0           0.0
Residents                             0           0.0
Temperature                         441         150.0
Timestamp                             0           0.0
Water_Consumption                     0           NaN
Water_Price                           0           0.0


In [12]:
# Fill missing 'Amenities' with 'Unknown'
train_df['Amenities'] = train_df['Amenities'].fillna('Unknown')
test_df['Amenities'] = test_df['Amenities'].fillna('Unknown')

In [13]:

train_df['Apartment_Type'] = train_df['Apartment_Type'].fillna(train_df['Apartment_Type'].mode()[0])
test_df['Apartment_Type'] = test_df['Apartment_Type'].fillna(train_df['Apartment_Type'].mode()[0])

train_df['Income_Level'] = train_df['Income_Level'].fillna(train_df['Income_Level'].mode()[0])
test_df['Income_Level'] = test_df['Income_Level'].fillna(train_df['Income_Level'].mode()[0])


In [14]:
for col in ['Appliance_Usage', 'Humidity', 'Temperature']:
    train_df[col] = train_df[col].fillna(train_df[col].median())
    test_df[col] = test_df[col].fillna(train_df[col].median())


In [15]:
from sklearn.preprocessing import OrdinalEncoder

label_cols = ['Apartment_Type', 'Income_Level', 'Amenities']
encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)

# Fit on train only
train_df[label_cols] = encoder.fit_transform(train_df[label_cols])
test_df[label_cols] = encoder.transform(test_df[label_cols])


In [16]:
from sklearn.preprocessing import LabelEncoder

label_cols = ['Apartment_Type', 'Income_Level', 'Amenities']
encoders = {}

for col in label_cols:
    # Convert both train and test to string
    train_df[col] = train_df[col].astype(str)
    test_df[col] = test_df[col].astype(str)

    # Handle unseen labels in test by setting them to "Unknown"
    train_labels = train_df[col].unique().tolist()
    test_df[col] = test_df[col].apply(lambda x: x if x in train_labels else "Unknown")

    # Add "Unknown" to training set (if not already there)
    if "Unknown" not in train_labels:
        train_df[col] = train_df[col].astype(str).replace("", "Unknown")
        train_df = pd.concat([train_df, pd.DataFrame({col: ["Unknown"]})], ignore_index=True)

    # Fit encoder and transform
    encoders[col] = LabelEncoder()
    train_df[col] = encoders[col].fit_transform(train_df[col])
    test_df[col] = encoders[col].transform(test_df[col])


In [17]:
train_df = train_df.drop(columns=['Timestamp'])
test_df = test_df.drop(columns=['Timestamp'])

In [18]:
X_train = train_df.drop(columns=['Water_Consumption'])
y_train = train_df['Water_Consumption']
X_test = test_df.copy()  # Test has no Water_Consumption


In [22]:
from sklearn.model_selection import train_test_split

X = train_df.drop(columns=['Water_Consumption'])
y = train_df['Water_Consumption']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [20]:
# Drop any rows where Water_Consumption is missing
train_df = train_df.dropna(subset=['Water_Consumption'])

# Redefine X and y
X = train_df.drop(columns=['Water_Consumption'])
y = train_df['Water_Consumption']


In [23]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

# Initialize and train
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_val)
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
score = max(0, 100 - rmse)

print("Validation RMSE:", rmse)
print("Custom Score:", score)



Validation RMSE: 20.171381049219562
Custom Score: 79.82861895078044


In [24]:
model.fit(X, y)
test_preds = model.predict(test_df)


In [25]:
submission = pd.DataFrame({
    "Timestamp": pd.read_csv(os.path.join(dataset_folder, "test.csv"))["Timestamp"],
    "Water_Consumption": test_preds
})

submission.to_csv("submission.csv", index=False)

# Create the text file (readme.txt)

In [28]:
explanation_text = """Smart Water Monitoring - Machine Learning Challenge
----------------------------------------------------

Author: Sreenija Earanki

Objective:
----------
To build a machine learning model that predicts daily water consumption per household using a variety of features such as household demographics, weather conditions, and behavioral patterns.

Dataset Summary:
----------------
- train.csv (14,000 rows): Contains historical water consumption data along with 11 features.
- test.csv (6,000 rows): Same features (except target).
- sample_submission.csv: Shows the expected format of the final predictions.

Target variable:
----------------
- Water_Consumption (float): Represents the total water consumed for a given 8-hour period.

Approach:
---------
1. Exploratory Data Analysis & Cleaning
   - Converted 'Timestamp' into datetime and extracted useful temporal features: `Hour`, `DayOfWeek`, `Month`.
   - Identified and handled missing values using appropriate imputation:
     - Categorical features (`Amenities`, `Apartment_Type`, `Income_Level`) filled with `'Unknown'` or mode.
     - Numeric features (`Humidity`, `Temperature`, `Appliance_Usage`) filled using median imputation.
   - Removed the original `Timestamp` column after extracting time features.

2. Feature Engineering
   - Extracted time-based features: `Hour`, `DayOfWeek`, and `Month` to capture daily and weekly patterns.
   - Applied **Label Encoding** to categorical features (`Apartment_Type`, `Income_Level`, `Amenities`) for compatibility with tree-based models.

3. Model Selection
   - Chose **Random Forest Regressor** as the baseline model due to its robustness, ability to handle nonlinear relationships, and resilience to outliers and multicollinearity.
   - Random Forest is particularly effective on tabular data and handles mixed data types (categorical + numerical) well after encoding.

4. Training & Validation
   - Split the training data (80/20) to evaluate model performance.
   - Used **Root Mean Squared Error (RMSE)** as the evaluation metric, consistent with the competition's custom scoring formula.
   - Achieved reasonable performance on validation set with limited hyperparameter tuning.

5. Prediction
   - Trained final model on the entire training dataset.
   - Generated predictions on the test set.
   - Constructed a submission file with predicted `Water_Consumption` values aligned with the correct `Timestamp` entries.

Tools & Libraries:
------------------
- Python 3.x
- pandas, numpy: Data manipulation and preprocessing
- scikit-learn: Model building and evaluation
  - RandomForestRegressor
  - LabelEncoder
  - train_test_split
- zipfile: For packaging submission files

Future Scope:
-------------------------
- Try more powerful models like XGBoost or LightGBM.
- Hyperparameter tuning using GridSearchCV or RandomizedSearchCV.
- Investigate temporal trends further with time series analysis techniques.
- Explore SHAP or permutation feature importance to interpret model behavior.

Submission Files:
-----------------
- submission.csv (prediction file)
- explanation.txt (this file)
- water.ipynb (source code)
"""

with open("explanation.txt", "w") as f:
    f.write(explanation_text)

In [29]:
import zipfile

with zipfile.ZipFile("source_files.zip", "w") as zipf:
    zipf.write("explanation.txt")
    zipf.write("submission.csv")